# Retail Demo Store Messaging Workshop - Braze

In this workshop we will use [Braze](https://braze.com/) to add the ability to personalize marketing messages to customers of the Retail Demo Store using customer behavioral data and the Personalize models you trained in the prior workshops. We are going to define a campaign in Braze to send target users an email with product recommendations from the Amazon Personalize campaign we created in the Personalization workshop earlier.  You will also create an email template using the Braze Connected Content feature.  When the campaign is launched, Connected Content will send emails to the users that belong in the target group.  These emails will be formatted using the Personalize Campaign you created earlier.

Recommended Time: 1 hour

## Prerequisites

This module uses Amazon Personalize to generate and associate personalized product recommendations for users.  The content of this workshop is presented with the assumption that you have either completed the [Personalization](../1-Personalization/1.1-Personalize.ipynb) workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

This part of the workshop requires access to a Braze account.  If you do not have a Braze account, please contact your Braze representative.  We also assume that you have completed at least the [Getting Started Braze LAB course](https://lab.braze.com/).

You will need to create a Braze API key for the duration of the workshop.  This API key will be used to create a set of temporary users for the workshop.  These temporary users will be deleted at the end of the workshop.

## Architecture

Before you set up Braze to send personalized messages to users, let's review the relevant parts of the Retail Demo Store architecture and how it uses Braze to integrate with the machine learning campaigns created in Personalize.

![Braze Personalization Architecture](images/braze-personalize-arch.png)

Braze will send emails to your users based on their behavior or based on attributes of their user profiles.  A discussion of Braze campaigns is beyond the scope of this document but Braze will ingest real-time events from mobile and web applications.  This data can be used to identify users and to build user profiles that can be used to determine when to message or email users.  

This event data flow will happen in parallel to the same behavioral event data being sent to Amazon Personalize. In this workshop, the demo store uses Amplify to send events to Personalize as shown in the diagram. This data is used to train a recommendations model that can in turn be used by Braze Connected Content to personalize content to users of your mobile and web applications when your Braze campaign runs.  

Braze Connected content will be able to get these recommendations via a recommendation service running in AWS.  Earlier in this workshop, this service was deployed using by the Cloud Formation templates you used to deploy the workshop environment in ECS.

### Building a Braze Email Template

Before you can run a campaign in Braze, you will need to create an email template that will be used to email your users.  This template will use the Braze Connected Content feature to fetch product recommendations for the users you will be emailing via your campaign.

First, you will need to get the name of the load balancer for the Retail Demo Store recommendations service you deployed at the beginning of the Retail Demo Store workshop.

The following code will query the AWS account you are using for this workshop, get a list of load balancers, and find the one that belongs to the Retail Demo Store and has the tag `recommendations`.  This URL will be stored in the `recommendations_elb_domain_name` variable after you run this code.

In [ ]:
import boto3

elbv2 = boto3.client('elbv2')
 
recommendations_elb_domain_name = None
 
elbs_paginator = elbv2.get_paginator('describe_load_balancers')
for elbs_page in elbs_paginator.paginate():
    for elb in elbs_page['LoadBalancers']:
        tags_response = elbv2.describe_tags(ResourceArns = [ elb['LoadBalancerArn'] ])
        for tag_desc in tags_response['TagDescriptions']:
            for tag in tag_desc['Tags']:
                if tag['Key'] == 'RetailDemoStoreServiceName' and tag['Value'] == 'recommendations':
                    recommendations_elb_domain_name = elb['DNSName']
                    break
            if recommendations_elb_domain_name:
                break
        if recommendations_elb_domain_name:
            break
    if recommendations_elb_domain_name:
        break
 
assert recommendations_elb_domain_name is not None, 'Unable to find Recommendations service ELB'
 
print('Recommendations DNS name: {}'.format(recommendations_elb_domain_name))

Next, you are going to prepare the HTML content for the email itself.  For simplicity, this workshop will only support HTML email templates.  The template provided with this workshop comes from the Braze sample templates, and can be easily adapted to other platforms if needed.

The following code will modify the `./braze-templates/braze-connected-content-email-template.html` template file in the workbook directory you found this notebook in.  This code will insert the correct Connected Content code for you, and will prepare a ZIP file which you will use in a moment to upload your template to your Braze campaign.  Make sure that you have your Jupyter directory tab open to the directory which contains this notebook.

In [ ]:
import zipfile

template_file_path = './braze-templates/braze-connected-content-email-template.html'
zip_file_path = './braze-templates/braze-connected-content-email-template.zip'

begin = '{% connected_content http://'
end = '/recommendations?userID={{${user_id}  | remove_first: "test_user_rds_"}}&fullyQualifyImageUrls=1&numResults=4 :save result %}'

connected_content_call = f'{begin}{recommendations_elb_domain_name}{end}'

text = open(template_file_path).read()
new_text = '\n'.join(connected_content_call if line.startswith('{% connected_content') else line for line in text.splitlines())
open(template_file_path, 'w').write(new_text)

zipfile.ZipFile(zip_file_path, mode='w').write(template_file_path)

Once the above code has completed, check that there is a `braze-connected-content-email-template.zip` in the `./braze-templates` directory in Jupyter. You will need this file when you configure a campaign in the following steps.  Once you have confirmed that this file is in the `./braze-templates` directory, download it to your computer:

![](./images/braze-jupyter-download.png)

### Anatomy of a Braze Connected Content Template

Before you create your campaign, let's quickly review how the Connected Content template you just built works.

The template is an HTML file that uses the [Braze Liquid](https://www.braze.com/docs/user_guide/personalization_and_dynamic_content/liquid/) templating language to format product recommendation results from a Braze Connected Content call.

The Connected Content call happens at the very top of the HTML template:

```
{% connected_content http://<Recommendation Service DNS Name>/recommendations?userID={{${user_id}  | remove_first: "test_user_rds_"}}&fullyQualifyImageUrls=1&numResults=4 :save result %}
```

This line tells Braze to call the Retail Demo Store recommendation REST service when building the email for the user specified in `user_id`.  This user ID is stripped of its `test_user_rds_` prefix using a Braze Liquid template function.  This is done because the Retail Demo Store uses integer user IDs, and the users you will upload will be created with string user identifiers prefixed with a workshop-specific identifier.  This was done for the workshop so that your Braze account does not encounter user ID conflicts with existing user IDs.  This is not strictly required for a production deployment of Amazon Personalize and Braze.

When the Connected Content call returns successfully, the JSON results returned from the recommendation service are stored in the variable `result` via the `:save` function.

### What Does the Recommendations Service Return?

In order to understand how the Connected Content template works, it is helpful to take a look at the responses that are returned from the recommendations service.  

The code below will invoke your recommendations service, and return a response for one of the users that exists in the Retail Demo Store user database.  You can change the `user_id` below to any valid ID.

If this code fails with `recommendations_elb_domain_name` not defined, you will need to re-run the recommendations service code two cells before this one.


In [ ]:
import requests
import json

user_id = 10
response = requests.get('http://{}/recommendations?userID={}&fullyQualifyImageUrls=1&numResults=4'.format(recommendations_elb_domain_name, user_id))
print(json.dumps(response.json(), indent = 2))


The return value from the recommendation service should look like:

```
[
  {
    "product": {
      "id": "2",
      "url": "http://recs.cloudfront.net/#/product/2",
      "sk": "",
      "name": "Striped Shirt",
      "category": "apparel",
      "style": "shirt",
      "description": "A classic look for the summer season.",
      "price": 9.99,
      "image": "http://recs.cloudfront.net/images/apparel/1.jpg",
      "featured": "true"
    }
  },
  {
    "product": {
      "id": "1",
      "url": "http://recs.cloudfront.net/#/product/1",
      "sk": "",
      "name": "Black Leather Backpack",
      "category": "accessories",
      "style": "bag",
      "description": "Our handmade leather backpack will look great at the office or out on the town.",
      "price": 109.99,
      "image": "http://recs.cloudfront.net/images/accessories/1.jpg",
      "featured": "true"
    }
  },

  ...
]
```

Setting up a similar service in your deployment of Amazon Personalize will give you the ability to surface product recommendations directly into Braze and other services that require real-time user personalization such as a mobile application or a web application.  

The recommendation service deployed in the workshop queries a Personalize campaign with the `user_id` that you provide.  Personalize will return a list of recommended product IDs for the user.  The service then looks up the catalog ID for each product and decorates the response with the product URL, image, name, etc.  These attributes should be available in your catalog management system.  In the case of the Retail Demo Store, there is a catalog service that provides (simulated) catalog items for the workshop.  The advantage of a service like this is that your target services like Braze do not need to do an additional lookup in order to get the information they require in order to surface a recommendation to a user, such as product name and the image associated with the product.


### How Does the Template Use the Returned Results?

Once the `result` variable is populated with the above JSON, the template can reference the product data in the variable using the Braze Liquid templating language.

A product cell in the template might look like this:

```
<table width="100%" border="0" cellspacing="0" cellpadding="0">
	<tr>
		<td width="115" class="text3" style="color:#212123; font-family:'Noto Sans', Arial,sans-serif; font-size:18px; line-height:22px; text-align:left">{{result[1].product.name}}</td>
		<td class="img" style="font-size:0pt; line-height:0pt; text-align:left" width="10"></td>
		<td class="text4" style="color:#6e6e70; font-family:'Noto Sans', Arial,sans-serif; font-size:12px; line-height:22px; text-align:right">{{result[1].product.price}}</td>
	</tr>
</table>
```

Note the `result[1].product.name` variable will be replaced with the name of the product that is first in the list of products returned, and the same goes for `result[1].product.price` for that product's price.  The same can be done for other product attributes such as the product's image.

### Creating a Braze Campaign

In a separate browser window or tab, log in to the Braze [console](https://dashboard.braze.com/auth).  You will need to have a Braze account configured for you to continue this workshop, or use your existing Braze account.

![Braze Login Screen](images/braze-create-template-0.png)

On the left hand pane of the Braze console, click the Templates & Media link.  This will show the Templates list screen.  Click the From File button under Basic Email Templates at the top of the screen.

![](images/braze-create-template-1.png)

Click the Upload from File button:

![](images/braze-create-template-2.png)

Navigate to the directory in which you downloaded the `braze-connected-content-email-template.zip` file you created earlier.  Click the Open or Save button to upload the ZIP file to Braze.

![](images/braze-create-template-3.png)

Braze will validate the file, and return a status window.  Click the Build Email button to continue.

![](images/braze-create-template-4.png)

On the next screen, give the template a name, such as `Braze Connected Content Workshop Template`.

![](images/braze-create-template-5.png)

On the bottom right of the screen, click the Save Template button.  

Make sure to click the Save button often, the editor will not auto-save your changes!

![](images/braze-create-template-6.png)

Scroll back up and click the Edit Email Body link

![](images/braze-create-template-7.png)

This will take you to the HTML template editor.  

At the very top, you should see the Connected Content call that calls the Personalize recommendation service, with the URL of your deployment of the recommendations service.

![](images/braze-create-template-8.png)

When you are finished looking at your template, click the Done button at the bottom right of the editor screen.

![](images/braze-create-template-9.png)

Before navigating away from the template editor, make sure to click the Save Template button on the bottom right of the template window, or your earlier changes will be lost.

![](images/braze-create-template-10.png)

### Create Braze User Profiles

Before you can run a campaign from Braze, you will need to create a set of demo user profiles with valid emails in Braze. 

For this workshop, we recommend using a set of test emails that (preferably) map to your own email address or to an email address that you can control.

The following code will use the Braze Users API to create 5 test users in your Braze account.  These users will all have a dummy user ID that starts with `test_user_rds`.  This will allow us to separate these profiles from other user IDs that you may have in Braze, and will allow us to easily delete the test users later.

For this code to work, you will need to get:

1. Your Braze endpoint URL, you can look this up [here](https://www.braze.com/docs/api/basics/#api-definitions).  Once you have your endpoint, set the `braze_endpoint_url` variable to the URL from the documentation.
2. Your Braze API key.  We recommend creating an API key just for this workshop, with the `users.track` and the `users.delete` permission at a minimum.  This key can be deleted after your are done with the workshop.  Change the `braze_api_key` variable below to your API key value.
3. Your email.  Your email account will need to support plus email addressing for this workshop to work correctly.  Input your email into the `your_email` variable below.

In [ ]:
import requests

braze_endpoint_url = '<YOUR BRAZE API ENDPOINT (SEE #1 ABOVE)>'
braze_api_key = '<YOUR BRAZE API KEY (SEE #2 ABOVE)'
your_email = '<YOUR EMAIL (SEE #3 ABOVE)>'

add_users = '/users/track'

tracked_demo_users = { 
    'attributes': []
}

def gen_email(num):
    email_parts = your_email.split('@')
    return email_parts[0] + f'+{num}@' + email_parts[1]

for id in range(10, 15):
    tracked_demo_users['attributes'].append({
        'external_id': f'test_user_rds_{id}',
        'first_name': f'Test User {id}',
        'email': gen_email(id)
    })

# Create some new users for the workshop in Braze
auth_header = {'Authorization': 'Bearer ' + braze_api_key}
response = requests.post(braze_endpoint_url + add_users, json=tracked_demo_users, headers=auth_header)
print(response.content)

This code should return a successful create message that looks like:

```
'{"attributes_processed":5,"message":"success"}'
```

As you saw in the template section earlier, Braze will pass a user ID to the recommendations service in order to get the recommendations for that user's email.  The user IDs you created now are going to be the user IDs that get passed to the recommendations service when the campaign is run with your email template.

## Create a Target User Segment

A user segment determines which users will receive emails from a campaign.

In the Braze console, click on Segments in the left pane, and then Create Segment in the top right corner of the window.

![](images/braze-create-segment-1.png)

Give your segment a name, like `Personalize Workshop`, and click the Create Segment button.

![](images/braze-create-segment-2.png)

Users can be matched to a segment by a variety of user attributes or behaviors.  For this workshop, you are going to use a regex that matches the external ID of the users you created in the previous section.

In the Filters section dropdown, scroll all the way to the bottom and select the external ID filter.  

Select matches regex for the matching criteria, and enter the regex `^test_user_rds` into the text window. 

Braze should update the bottom of the window with a segment user count of 5.  This is the number of users the user create script created for you.

Click the Save button. 

![](images/braze-create-segment-3.png)

### Create and Run a Braze Campaign

In order to send personalized emails, you will need to configure a campaign.  

In the Braze console, click the Campaigns link in the left hand menu pane. 

![](images/braze-campaign-1.png)

Select Create Campaign in the upper right corner of the screen.

In the dropdown that appears, select Email for the campaign type.

For a more complete discussion of different Braze campaign types, check out their [campaign docs](https://www.braze.com/docs/user_guide/engagement_tools/campaigns/).

![](images/braze-campaign-2.png)

Enter the name of the campaign - something like `Braze Personalize Campaign`.

![](images/braze-campaign-3.png)

After entering the campaign name, scroll down to the bottom of the screen and select the Connected Content email template you created earlier in the workshop. 

![](images/braze-campaign-4.png)

After you select your template, scroll up and click the Edit Sending Info link.  

Enter a subject line for your email.

Braze will not let you initiate a campaign without a subject line.

![](images/braze-campaign-5.png)

Click the Done button when you are done entering the subject line.

![](images/braze-campaign-6.png)

For this campaign, you will want emails to send immediately after the campaign launches, so select 'Send as soon as campaign is launched' under Time Based Scheduling Options.

![](images/braze-campaign-7.png)

Scroll down to the bottom of this pane.  This is optional but it may help in debugging any issues with getting emails from this campaign.  Under Delivery Controls, you will want to specify a short time period for when users in this campaign can be contacted again via email.  

Braze will use this setting to prevent real users from being spammed by campaigns. In this workshop you are using your own email, so this should not prevent you from re-sending the campaign again if necessary.

Click the Forward link at the bottom of the screen.

![](images/braze-campaign-8.png)

Under Target Users, select the target user segment you created in the last step, then click the Forward link at the bottom of the screen.

![](images/braze-campaign-9.png)

You will not be using conversion events for this campaign, so you can click the Forward link on this screen.

![](images/braze-campaign-10.png)

At this point, you are ready to kick off the campaign.

Click the Launch Campaign button.  This will immediately start sending emails to the accounts you created earlier.

![](images/braze-campaign-11.png)

You should see an email in your inbox that looks something like this:

![](images/braze-email-example.jpg)

## Workshop Complete

Congratulations! You have completed the Retail Demo Store Braze Workshop.

### Cleanup

The following code will delete the users you created in your Braze account so that they do not remain in your account.


In [ ]:
delete_users = '/users/delete'

delete_user_list = {
    'external_ids': [profile['external_id'] for profile in tracked_demo_users['attributes']]
}

response = requests.post(braze_endpoint_url + delete_users, json=delete_user_list, headers=auth_header)
print(response.content)

You may choose to delete the Braze template, segment, and campaign as well, via the Braze console.

If you launched the Retail Demo Store in your personal AWS account **AND** you are done with all workshops & your evaluation of the Retail Demo Store, you can remove all provisioned AWS resources and data by deleting the CloudFormation stack you used to deploy the Retail Demo Store.

> If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the following cleanup steps unless otherwise instructed.